In [1]:
# general
from datetime import datetime
from dotenv import load_dotenv, find_dotenv
import os

# custom local libs
from function import baseball_lambda

/Users/rosclay/Documents/VSCode/langgraph-multi-agent/venv/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: The function `bind_tools` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
Python REPL can execute arbitrary code. Use with caution.


In [2]:
# read local .env file
_ = load_dotenv(find_dotenv()) 

In [3]:
# set Langsmith project
today = datetime.now().strftime("%Y%m%d")
os.environ["LANGCHAIN_PROJECT"] = f"Baseball Curveballs - {today}"

In [4]:
session_id = '88241'

In [5]:
task = "Consider the first week of June 2019 - find 3 pitchers who's changeups were most similar to Justin Verlander's"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Retrieve Node ***

Distance to neareast plan: 0.4059051275253296

*** Entered Modify Node ***

Modifying nearest plan with User input
The revised plan to find the 3 pitchers with changeups most similar to Justin Verlander's in the first week of June 2019 is as follows:

1. Use playerid_lookup to find Justin Verlander's MLBAM player ID.

2. Use statcast_pitcher with the following parameters to get Verlander's changeup data for the first week of June 2019:
   - start_dt = '2019-06-01'
   - end_dt = '2019-06-07'
   - player_id = Justin Verlander's MLBAM ID

3. From the returned data, calculate the average release_speed, release_spin_rate, pfx_x, and pfx_z for Verlander's changeups. Store these 4 values in a "Verlander changeup average vector".

4. Use statcast with the following parameters to get pitching data for all pitchers in the first week of June 2019:
   - start_dt = '2019-06-01'
   - end_dt = '2019-06-07'

5. Filter the results to onl

In [6]:
task = "looks good"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***
Known plan. Executing with Sonnet

*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
import pandas as pd
from pybaseball import playerid_lookup, statcast, statcast_pitcher, playerid_reverse_lookup
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
verlander_id = playerid_lookup('verlander', 'justin')['key_mlbam'].values[0]
print(f"Justin Verlander's MLBAM player ID is: {verlander_id}")

Result:
Gathering player lookup table. This may take a moment.
Justin Verlander's MLBAM player ID is: 434378


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
verlander_changeups = statcast_pitcher('2019-06-01', '2019-06-07', player_id=verlander_id)

Result:
Gathering Player Data


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
verlander_changeups = verlander_changeups[verland

  0%|          | 0/7 [00:00<?, ?it/s]


*** Entered Execute Node ***


Executing:
all_pitches = statcast('2019-06-01', '2019-06-07')


100%|██████████| 7/7 [00:12<00:00,  1.78s/it]



Result:
This is a large query, it may take a moment to complete


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
print(f"Retrieved {len(all_pitches)} pitches from 2019-06-01 to 2019-06-07")

Result:
Retrieved 27211 pitches from 2019-06-01 to 2019-06-07


*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
changeups = all_pitches[all_pitches['pitch_type'] == 'CH']

*** Entered Generate Node ***


*** Entered Execute Node ***


Executing:
pitcher_averages = changeups.groupby('pitcher').agg({'release_speed': 'mean',
                                                     'release_spin_rate': 'mean', 
                                                     'pfx_x': 'mean',
                                                     'pfx_z': 'mean'}).reset_index()

print(f"Calculated average changeup metrics for {len(pitcher_averages)} pitchers")

Result:
Calculated average changeup metrics for 253 pitchers


*** Entered Generate Node ***


*** Entered E

In [7]:
task = "yes"
result = baseball_lambda.execute_workflow(task, session_id)
print(result)


*** Entered Orchestrate Node ***

*** Entered Memorize Node ***

Thank you, task has been commited to memory
